# 필요한 라이브러리 로드

In [22]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import  mean_squared_error ,mean_absolute_percentage_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor, VotingRegressor 
from sklearn.ensemble import StackingRegressor

# 전처리 된 풍력 데이터 로드

## 풍력 데이터 일 단위분석

In [2]:
df = pd.read_csv('./data/wind.csv') # 시간 단위 데이터 로드
df['일시'] = pd.to_datetime(df['일시']) # 데이터 타입을 날짜 형식으로 변환
df['년'] = df['일시'].dt.year
df['월'] = df['일시'].dt.month
df['일'] = df['일시'].dt.day
df_1 = df.drop('일시', axis = 1 )

result = df_1.groupby(['년', '월', '일']).agg({
    ' 풍력 발전량(MWh) ': 'sum',
    '기온(°C)': 'mean',
    '풍향(deg)': 'mean',
    '풍속(m/s)': 'mean',
    '강수량(mm)': 'mean',
    '습도(%)': 'mean'
}).reset_index() # 시간단위로 기후 데이터는 평균, 풍력 발전량은 총합을 사용

result = result[[' 풍력 발전량(MWh) ', '기온(°C)','풍속(m/s)','강수량(mm)', '습도(%)']] # 사용할 컬럼을 선택

In [3]:
# Decision Tree
def DT_anaytics(X_train, X_test,y_train, y_test):    
    #Decision Tree 객체 생성
    DT = DecisionTreeRegressor()
    #GridSearch 에 사용할 그리드의 범위 설정
    params = {'max_depth': [5], 'min_samples_split': [100, 1000, 10000, 100000]}
    # 5-fold교차 검증, 모델 평가 지표로 neg_mean_squared_error 사용
    DT_search = GridSearchCV(DT, params, cv=5, scoring='neg_mean_squared_error')
    # GridSearch 의 결과로 사용된 하이퍼 파라미타가 적용된 DT 분석 진행
    DT_search.fit(X_train,y_train) # 학습
    best_clf = DT_search.best_estimator_ # 최적의 하이퍼 파라미터
    y_pred = DT_search.predict(X_test) #예측

    # 성능 지료 rmse, mse, mape사용
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    mape= mean_absolute_percentage_error(y_test, y_pred)
    print(f'''
rmse for Decision Tree :{rmse}
mse for Decision Tree :{mse}
mape for Decision Tree :{mape}
''')

def RF_anaytics(X_train, X_test,y_train, y_test):    
    # RandomForestRegressor객체 생성, 깊이 20
    rfc = RandomForestRegressor(max_depth=20, random_state=24)
    rfc.fit(X_train,y_train) # 학습
    y_pred = rfc.predict(X_test) # 예측

    # 성능 지료 rmse, mse, mape사용
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    mape= mean_absolute_percentage_error(y_test, y_pred)
    print(f'''
rmse for RF_anaytics :{rmse}
mse for RF_anaytics :{mse}
mape for Decision Tree :{mape}
''')


def MLP_anaytics(X_train, X_test,y_train, y_test): 
    # MLPRegressor 객체 생성 은닉층3개, 1500 iterations,30% dorpout 설정
    regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1500, random_state=42, alpha=0.03)
    regr.fit(X_train, y_train)
    regr.predict(X_test)
    y_pred = regr.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    mape= mean_absolute_percentage_error(y_test, y_pred)
    # 성능 지료 rmse, mse, mape사용
    print(f'''
rmse for MLP_anaytics :{rmse}
mse for MLP_anaytics :{mse}
mape for MLP_anaytics :{mape}
''')
    
    
def ensemble(X_train, X_test, y_train, y_test):
    # Individual regressors
    rf_regressor = RandomForestRegressor(max_depth=20, random_state=24)
    mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=1500, random_state=42, alpha=0.03)

    # Ensemble regressor
    ensemble_regressor = VotingRegressor(estimators=[
        ('rf', rf_regressor),
        ('mlp', mlp_regressor)
    ])

    ensemble_regressor.fit(X_train, y_train)
    y_pred = ensemble_regressor.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(f'''
rmse for Ensemble :{rmse}
mse for Ensemble :{mse}
mape for Ensemble :{mape}
''')

In [5]:
predicts = [' 풍력 발전량(MWh) ',' 태양광 발전량(MWh) ']

wind_1 = result[[' 풍력 발전량(MWh) ',  '풍속(m/s)', '습도(%)']].dropna()

X_train, X_test, y_train, y_test = train_test_split(wind_1.drop(predicts[0], axis=1),wind_1[predicts[0]], random_state= 24)
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on your data and transform it
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## 예측결과 성능 지표 출력

In [6]:
DT_anaytics(X_train, X_test,y_train, y_test)
RF_anaytics(X_train, X_test,y_train, y_test)
MLP_anaytics(X_train, X_test,y_train, y_test)
ensemble(X_train, X_test,y_train, y_test)


rmse for Decision Tree :4188.706205930472
mse for Decision Tree :17545259.679600447
mape for Decision Tree :0.6511812624846266


rmse for RF_anaytics :4044.533914603062
mse for RF_anaytics :16358254.586374369
mape for Decision Tree :0.6041592638946877


rmse for MLP_anaytics :4095.0252293996136
mse for MLP_anaytics :16769231.629419357
mape for MLP_anaytics :0.6493066231895852


rmse for Ensemble :3952.9225147815964
mse for Ensemble :15625596.407867262
mape for Ensemble :0.6132867450168387


rmse for Stacking Ensemble :4020.29485138817
mse for Stacking Ensemble :16162770.692098228
mape for Stacking Ensemble :0.61029315345218



## 시간단위 분석 진행

In [ ]:
df_hour = pd.read_csv('./data/wind.csv')
df_hour = df_hour.drop(['일시'], axis =1 )

In [ ]:
df_hour

,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),풍력 발전량(MWh),풍향(deg)_0,풍향(deg)_1,풍향(deg)_2,풍향(deg)_3,...,_경기도,_경상남도,_경상북도,_부산시,_울산시,_인천시,_전라남도,_전라북도,_제주도,_충청남도
0,-8.2,0.0,0.3,0.0,77.0,0.00,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-8.6,0.0,0.0,0.0,79.0,15.14,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-7.9,284.8,0.7,0.0,78.0,20.26,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-7.8,215.2,1.4,0.0,80.0,12.27,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-6.7,209.7,1.4,0.0,78.0,9.23,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271780,3.6,322.3,1.4,0.0,45.0,0.00,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
271781,3.1,296.8,2.7,0.0,46.0,0.00,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
271782,3.0,290.7,2.7,0.0,45.0,0.00,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
271783,2.7,264.6,2.5,0.0,42.0,0.00,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
def DT_anaytics(X_train, X_test,y_train, y_test):    
    DT = DecisionTreeRegressor()
    params = {'max_depth': [5], 'min_samples_split': [100, 1000, 10000, 100000]}
    DT_search = GridSearchCV(DT, params, cv=5, scoring='neg_mean_squared_error')
    DT_search.fit(X_train,y_train)
    best_clf = DT_search.best_estimator_
    y_pred = DT_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
# Calculate MAPE
    print(f'''
rmse for Decision Tree :{rmse}
mse for Decision Tree :{mse}
''')


def RF_analytics(X_train, X_test, y_train, y_test):
    # RandomForestRegressor 모델 정의
    rfc = RandomForestRegressor(random_state=24)

    # 탐색할 하이퍼파라미터 그리드 정의
    param_grid = {'max_depth': [10,15,20]}

    # GridSearchCV 객체 생성
    grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='neg_mean_squared_error')
    
    # Grid Search를 사용하여 모델 훈련
    grid_search.fit(X_train, y_train)

    # 최적의 모델 추출
    best_rfc = grid_search.best_estimator_

    # 테스트 데이터로 예측 및 성능 평가
    y_pred = best_rfc.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)

    print(f'''
rmse for RF_analytics: {rmse}
mse for RF_analytics: {mse}
Best Parameters for RF_analytics: {grid_search.best_params_}
''')

def MLP_analytics(X_train, X_test, y_train, y_test):
    # MLPRegressor 모델 정의
    regr = MLPRegressor(max_iter=1000, random_state=42)

    # 탐색할 하이퍼파라미터 그리드 정의
    param_grid = {
        'hidden_layer_sizes': [(100, 100, 100), (50, 50, 50)],  
        'alpha': [0.01, 0.03, 0.1]
    }

    # GridSearchCV 객체 생성
    grid_search = GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error')

    # Grid Search를 사용하여 모델 훈련
    grid_search.fit(X_train, y_train)

    # 최적의 모델 추출
    best_regr = grid_search.best_estimator_

    # 테스트 데이터로 예측 및 성능 평가
    y_pred = best_regr.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)

    print(f'''
rmse for MLP_analytics: {rmse}
mse for MLP_analytics: {mse}
Best Parameters for MLP_analytics: {grid_search.best_params_}
''')



In [20]:
predicts = [' 풍력 발전량(MWh) ',' 태양광 발전량(MWh) ']

wind_1 = df_hour.dropna()

X_train, X_test, y_train, y_test = train_test_split(wind_1.drop(predicts[0], axis=1),wind_1[predicts[0]], random_state= 24)
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on your data and transform it
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

DT_anaytics(X_train, X_test,y_train, y_test)
RF_anaytics(X_train, X_test,y_train, y_test)
MLP_anaytics(X_train, X_test,y_train, y_test)


rmse for Decision Tree :44.7198844252033
mse for Decision Tree :1999.8680630035403


rmse for RF_anaytics :40.35644669998307
mse for RF_anaytics :1628.642790248574


rmse for MLP_anaytics :42.18982812943884
mse for MLP_anaytics :1779.9815975915888



# 2023년 1월 1일에 대한 예측 진행

In [27]:
predicts = [' 풍력 발전량(MWh) ',' 태양광 발전량(MWh) ']

wind_1 = result[[' 풍력 발전량(MWh) ', '기온(°C)', '풍속(m/s)','강수량(mm)', '습도(%)']].dropna()

X_train, X_test, y_train, y_test = train_test_split(wind_1.drop(predicts[0], axis=1),wind_1[predicts[0]], random_state= 24)
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on your data and transform it
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


# Fit the scaler on your data and transform it
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
rfc = RandomForestRegressor(max_depth=20, random_state=24)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mape= mean_absolute_percentage_error(y_test, y_pred)

asdf = pd.DataFrame({'기온(°C)':0.3375, '풍속(m/s)': 2.65, '강수량(mm)': 0, '습도(%)': 55.33}, index=[0])


# Select only the relevant features from asdf that match X_test
new_row = asdf.iloc[0, :].values

# Add a single row to X_test
X_test_with_new_row = np.vstack([X_test, new_row])
a = rfc.predict(X_test_with_new_row)
print(a[-1])

15867.666200000005
